- competition/dataset : [https://www.kaggle.com/c/spooky-author-identification](https://www.kaggle.com/c/spooky-author-identification)
- date : 2021/03/10
- original : [https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle)

## Approaching (Almost) Any NLP Problem on Kaggle

**✏ 필사 1회** 

이 포스팅에서는 Kaggle의 NLP 문제에 대해 이야기하고자 합니다. 예제로 우리는 이 대회에서 주어진 데이터를 사용합니다. 먼저 기본 모델을 만들고, 다른 feature들을 이용하여 개선합니다. 또한 DNN이 어떻게 사용되는지 확인하고, 일반적인 앙상블에 대한 몇 가지 아이디어로 포스팅을 마무리할 예정입니다.  

**This covers:**  
* tfidf
* count features
* logistic regression
* naive bayes
* svm
* xgboost
* grid search
* word vectors
* LSTM
* GRU
* Ensembling

In [15]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm_notebook
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [3]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [5]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


이 문제는 텍스트에 주어진 EAP, HPL, MWS 저자를 예측해야 합니다. 간단히 말해서, 텍스트는 세 가지의 클래스로 분류되어 있습니다.  

이 문제에 대해 Kaggle은 multi-class log-loss를 평가 메트릭으로 지정했습니다. 이는 다음과 같은 방법으로 구현됩니다: [https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py](https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)

In [6]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    '''
    log loss 메트릭의 멀티 클래스 버전
    :param actual: 실제 타겟 클래스를 포함한 array
    :param predicted: 클래스 예측 값이 포함된 matrix, 한 클래스당 하나의 확률
    '''
    # actual이 binary array가 아니면 변환
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

text label을 정수형 0, 1, 2로 변환하기 위해 scikit-learn의 `LabelEncoder`를 사용합니다.

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train['author'].values)

더 진행하기 앞서 훈련 데이터 셋과 검증 데이터 셋을 분리하는 것이 중요합니다. scikit-learn 모듈의 `model_selection`의 `train_test_split`을 사용하여 실행할 수 있습니다.

In [8]:
xtrain, xvalid, ytrain, yvalid = train_test_split(
    train['text'].values, y, stratify=y, random_state=42,
    test_size=0.1, shuffle=True
)

In [10]:
print(xtrain.shape)
print(xvalid.shape)

(17621,)
(1958,)


### Building Basic Models
첫 번째 모델을 생성해봅시다.  

우리의 첫 모델은 TF-IDF(Tearm Frequency - Inverse Document Frequency)와 간단한 로지스틱 회귀입니다.

In [13]:
# 항상 이 feature들로 시작
tfv = TfidfVectorizer(
    min_df=3, max_features=None, strip_accents='unicode',
    analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 3),
    use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english'
)

# training과 test 셋에 TF-IDF fitting (준지도학습)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [16]:
# TFIDF에 로지스틱 회귀 적용
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

logloss: 0.572


첫 번째 모델의 multiclass logloss는 0.572입니다.  

동일한 모델을 다른 데이터로 살펴보겠습니다.  

TF-DIF 대신에 단어 수를 feature로 사용할 수 있습니다. scikit-learn의 `CountVectorizer`을 사용하면 간단하게 해결할 수 있습니다.

In [17]:
ctv = CountVectorizer(
    analyzer='word', token_pattern=r'\w{1,}',
    ngram_range=(1, 3), stop_words='english'
)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv = ctv.transform(xtrain)
xvalid_ctv = ctv.transform(xvalid)

In [19]:
# Counts에 로지스틱 회귀 적용
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

logloss: 0.527


0.048 개선되었습니다.  

다음으로, 예전에 꽤 유명했던 Naive Bayes 모델을 시도해보겠습니다.

In [20]:
# TFIDF에 Naive Bayes 적용
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

logloss: 0.578


Counts에 로지스틱 회귀를 적용한 것이 더 성능이 좋습니다. 위 모델을 counts data에 사용하면 어떻게 될까요?

In [22]:
# Counts에 Naive Bayes 적용
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

logloss: 0.485


오래된 것이 더 잘 작동합니다!  

오래된 알고리즘 중에 SVM이 있으며, 일부 사람들은 SVM을 매우 사랑합니다. 따라서, 데이터 셋에 SVM을 시도해보겠습니다.  

SVM은 시간이 오래 걸리기 때문에 Singular Value Decomposition을 사용하여 TF-IDF의 feature의 수를 줄이겠습니다.  

SVM을 적용하기 전에 반드시 표준화를 해야 합니다.

In [23]:
# SVD를 적용하여 120개의 컴포넌트 선택
# SVM 모델에는 120 - 200개의 컴포넌트가 적절함
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# SVD로 얻은 데이터 스케일링
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

이제 SVM을 적용해보겠습니다.

In [26]:
# SVM fitting
# 오!래!걸!림!
clf = SVC(C=1.0, probability=True) # 확률 값을 구하는 것이기 때문
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

logloss: 0.725


성능이 좋지 않습니다. 

또 인기있는 알고리즘 중 하나인 xgboost를 적용해보겠습니다.

In [27]:
# TF-IDF에 xgboost 적용
clf = xgb.XGBClassifier(
    max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8,
    nthread=10, learning_rate=0.1
)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ('logloss: %0.3f'% multiclass_logloss(yvalid, predictions))

[09:29:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.781


In [28]:
# Counts에 xgboost 적용
clf = xgb.XGBClassifier(
    max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8,
    nthread=10, learning_rate=0.1
)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ('logloss: %0.3f'% multiclass_logloss(yvalid, predictions))

[09:29:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.772


In [29]:
# TF-IDF svd feature에 xgboost 적용
clf = xgb.XGBClassifier(
    max_depth=7, n_estimators=200, colsample_bytree=0.8, subsample=0.8,
    nthread=10, learning_rate=0.1
)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ('logloss: %0.3f'% multiclass_logloss(yvalid, predictions))

[09:31:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.764


In [30]:
# TF-IDF svd feature에 xgboost 적용
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ('logloss: %0.3f'% multiclass_logloss(yvalid, predictions))

[09:32:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.790


### Grid Search
이것은 하이퍼파라미터 최적화를 위한 기술입니다. 효과적이지는 않지만 사용하고자 하는 그리드를 알고 있다면 좋은 결과를 얻을 수 있습니다. 다음 포스트에 일반적으로 사용해야 할 파라미터에 대해 정리해 두었습니다: [http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/](http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/)  

이 섹션에서는 로지스틱 회귀를 사용한 grid search에 대해 이야기하겠습니다.  

grid search를 시작하기 전에 먼저 스코어링 함수를 만들어야 합니다. scikit-learn의 `make_scorer` 함수를 사용하여 수행할 수 있습니다.

In [31]:
mll_scorer = metrics.make_scorer(
    multiclass_logloss, greater_is_better=False, needs_proba=True
)

다음으로 파이프라인이 필요합니다. 시연을 위해 SVD, 스케일링, 로지스틱 회귀로 구성된 파이프라인을 사용합니다. 많은 모듈들을 파이프라인에 적용하여 이해하는 것이 하나만 사용하는 것보다 더 낫습니다.

In [32]:
# SVD 초기화
svd = TruncatedSVD()

# Standard Scaler 초기화
scl = preprocessing.StandardScaler()

# 로지스틱 회귀 초기화
lr_model = LogisticRegression()

# 파이프라인 생성
clf = pipeline.Pipeline([('svd', svd), ('scl', scl), ('lr', lr_model)])

파라미터 그리드가 필요합니다.

In [33]:
param_grid = {'svd__n_components':[120, 180],
              'lr__C':[0.1, 1.0, 10],
              'lr__penalty':['l1', 'l2']}

SVD는 120개와 180개의 컴포넌트를 평가하고, 로지스틱 회귀의 경우 l1과 l2 패널티로 세 가지 다른 C의 값들을 평가합니다. 이제 이 파라미터들로 grid search를 시작할 수 있습니다.

In [35]:
# Grid Search 모델 초기화
model = GridSearchCV(
    estimator=clf, param_grid=param_grid, scoring=mll_scorer,
    verbose=10, n_jobs=1, refit=True, cv=2
)

# fitting
model.fit(xtrain_tfv, ytrain) # 전체 데이터를 사용해도 됨
print('Best score: %0.3f'%model.best_score_)
print('Best parameters set:')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s: %r'%(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV 1/2; 1/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=120...........
[CV 1/2; 1/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=120; total time=   1.2s
[CV 2/2; 1/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=120...........
[CV 2/2; 1/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=120; total time=   1.0s
[CV 1/2; 2/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=180...........
[CV 1/2; 2/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=180; total time=   1.7s
[CV 2/2; 2/12] START lr__C=0.1, lr__penalty=l1, svd__n_components=180...........
[CV 2/2; 2/12] END lr__C=0.1, lr__penalty=l1, svd__n_components=180; total time=   1.5s
[CV 1/2; 3/12] START lr__C=0.1, lr__penalty=l2, svd__n_components=120...........
[CV 1/2; 3/12] END lr__C=0.1, lr__penalty=l2, svd__n_components=120; total time=   0.9s
[CV 2/2; 3/12] START lr__C=0.1, lr__penalty=l2, svd__n_components=120...........
[CV 2/2; 3/12

점수는 SVM과 비슷합니다. 이 기술은 다음과 같이 xgboost나 다항 naive bayes를 정교하게 튜닝하는데 사용될 수 있습니다. 여기서는 tfidf 데이터를 사용합니다.

In [39]:
nb_model = MultinomialNB()

# 파이프라인 생성
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha':[0.001, 0.01, 0.1, 1, 10, 100]}

# Grid Search 모델 초기화
model = GridSearchCV(
    estimator=clf, param_grid=param_grid, scoring=mll_scorer,
    verbose=10, n_jobs=-1, refit=True, cv=2
)

# fitting
model.fit(xtrain_tfv, ytrain) # 전체 데이터를 사용해도 됨
print('Best score: %0.3f'%model.best_score_)
print('Best parameters set:')
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print('\t%s: %r'%(param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.492
Best parameters set:
	nb__alpha: 0.1


NLP 문제에서는 일반적으로 단어 벡터를 살펴봅니다. 단어 벡터는 데이터에 대한 많은 인사이트를 제공합니다. 살펴봅시다.

### word Vectors
깊게 들어가지 않고, 어떻게 문장 벡터를 만들고 이것을 머신러닝 모델에 어떻게 사용하는지에 대해 설명해보겠습니다. 저는 GloVe vertors, word2vec, fasttext를 굉장히 좋아합니다. 이 포스트에서는 GloVe vectors를 사용할 것입니다. 이것은 [http://www-nlp.stanford.edu/data/glove.840B.300d.zip](http://www-nlp.stanford.edu/data/glove.840B.300d.zip)에서 다운받을 수 있습니다.

In [52]:
# 딕셔너리에 GloVe vector 로드
embeddings_index = {}
f = open('data/glove.840B.300d.txt', encoding='utf-8')
for line in tqdm_notebook(f):
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        f.__next__()
f.close()

print('Found %s word vectors.'%len(embeddings_index))

0it [00:00, ?it/s]

Found 2195864 word vectors.


In [55]:
# 전체 문장에 대해 표준화된 벡터를 생성하는 함수
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt(v**2).sum()

In [56]:
# training과 validation 데이터 셋에 대해 위 함수를 적용하여 문장 벡터 생성
xtrain_glove = np.array([sent2vec(x) for x in tqdm_notebook(xtrain)])
xvalid_glove = np.array([sent2vec(x) for x in tqdm_notebook(xvalid)])

  0%|          | 0/17621 [00:00<?, ?it/s]

  0%|          | 0/1958 [00:00<?, ?it/s]

glove feature에 대해 xgboost의 성능을 확인해봅시다.

In [57]:
# glove feature에 xgboost fitting 1
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

[13:07:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:07:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.720


In [58]:
# glove feature에 xgboost fitting 2
clf = xgb.XGBClassifier(
    max_depth=7, n_estimators=200, colsample_bytree=0.8,
    subsample=0.8, nthread=10, learning_rate=0.1, silent=False
)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)
print('logloss: %0.3f'%multiclass_logloss(yvalid, predictions))

[13:09:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:09:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
logloss: 0.679


### Deep Learning
여기서는 GloVe feature에 대해 LST과 간단한 Dense Network를 훈련시킬 것입니다. 먼저 dense network를 시작해보겠습니다.

In [59]:
# 신경망 전에 데이터 스케일링
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [60]:
# 신경망을 위해 레이블을 이분화
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [61]:
# 3 layer sequential neural net 생성
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [62]:
model.fit(
    xtrain_glove_scl, y=ytrain_enc, batch_size=64, epochs=5, verbose=1,
    validation_data=(xvalid_glove_scl, yvalid_enc)
)

Epoch 1/5
276/276 [==============================] - 1s 5ms/step - loss: 0.9170 - val_loss: 0.7175
Epoch 2/5
276/276 [==============================] - 1s 4ms/step - loss: 0.7066 - val_loss: 0.6817
Epoch 3/5
276/276 [==============================] - 1s 4ms/step - loss: 0.6323 - val_loss: 0.6678
Epoch 4/5
276/276 [==============================] - 1s 4ms/step - loss: 0.5910 - val_loss: 0.6729
Epoch 5/5
276/276 [==============================] - 1s 4ms/step - loss: 0.5594 - val_loss: 0.6652


더 나은 결과를 위해 계속 파라미터를 튜닝하고, 레이어를 추가하고, dropout을 높여야 합니다. 여기서는 단지 최적화 없이 xgboost보다 구현과 실행이 빠르고 더 나은 결과를 얻을 수 있다는 것을 보여드리고자 합니다.  

LSTM과 같은 방법을 사용하기 위해서는 텍스트 데이터를 tokenize해야 합니다.

In [65]:
# keras의 tokenizer 사용
token = text.Tokenizer(num_words=None)
max_len = 70
token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [66]:
# 데이터 셋에 있는 단어들에 대한 임베딩 매트릭스 생성
embedding_matrix = np.zeros((len(word_index)+1, 300))
for word, i in tqdm_notebook(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

  0%|          | 0/25943 [00:00<?, ?it/s]

In [67]:
# glove embedding과 2 dense layer로 이루어진 간단한 LSTM
model = Sequential()

model.add(
    Embedding(len(word_index)+1, 300, weights=[embedding_matrix],
              input_length=max_len, trainable=False)
)
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# fitting
model.fit(
    xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1,
    validation_data=(xvalid_pad, yvalid_enc)
)

Epoch 1/100
35/35 [==============================] - 40s 1s/step - loss: 1.0676 - val_loss: 0.9156
Epoch 2/100
35/35 [==============================] - 40s 1s/step - loss: 0.8932 - val_loss: 0.7366
Epoch 3/100
35/35 [==============================] - 43s 1s/step - loss: 0.8076 - val_loss: 0.6992
Epoch 4/100
35/35 [==============================] - 42s 1s/step - loss: 0.7796 - val_loss: 0.6754
Epoch 5/100
35/35 [==============================] - 45s 1s/step - loss: 0.7553 - val_loss: 0.6582
Epoch 6/100
35/35 [==============================] - 45s 1s/step - loss: 0.7339 - val_loss: 0.6946
Epoch 7/100
35/35 [==============================] - 46s 1s/step - loss: 0.7249 - val_loss: 0.6500
Epoch 8/100
35/35 [==============================] - 42s 1s/step - loss: 0.7046 - val_loss: 0.6225
Epoch 9/100
35/35 [==============================] - 43s 1s/step - loss: 0.6760 - val_loss: 0.6068
Epoch 10/100
35/35 [==============================] - 45s 1s/step - loss: 0.6666 - val_loss: 0.5963
Epoch 11/

'왜 dropout을 이렇게나 많이 할까?'라는 질문을 할 수 있습니다. dropout을 하지 않거나 적은 상태로 모델을 fitting하면 과적합이 발생합니다.  

양방향 LSTM이 더 나은 결과를 줄 수 있는지 확인해보겠습니다. keras와 함께하면 식은 죽 먹기입니다.

In [ ]:
# glove embedding과 2 dense layer로 이루어진 간단한 양방향 LSTM
model = Sequential()

model.add(
    Embedding(len(word_index)+1, 300, weights=[embedding_matrix],
              input_length=max_len, trainable=False)
)
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# fitting
earlystop = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto'
)
model.fit(
    xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1,
    validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop]
)

 두 레이어로 이루어진 GRU도 사용해보겠습니다.

In [ ]:
# glove embedding과 2 dense layer로 이루어진 간단한 GRU
model = Sequential()

model.add(
    Embedding(len(word_index)+1, 300, weights=[embedding_matrix],
              input_length=max_len, trainable=False)
)
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# fitting
earlystop = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto'
)
model.fit(
    xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1,
    validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop]
)

### Ensembling